In [16]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
#https://www.morningstar.co.jp/etf_foreign/search.jsp
# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)  #toyota
# data = web.DataReader('1321.T', 'yahoo', start, end)  #(NEXT FUNDS)日経225連動型上場投信
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('1655.T', 'yahoo', start, end)  #iシェアーズ S&P500 米国株 ETF　→　取得されるデータがおかしい？？
# data = web.DataReader('2558.T', 'yahoo', start, end)  #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)  #MAXIS ナスダック100上場投信
data = web.DataReader('2559.T', 'yahoo', start, end)    #MAXIS 全世界株式(オール・カントリー)上場投信

# data = web.DataReader('BTC-JPY', 'yahoo', start, end)   #Bitcoin
data = data.astype("double")
print(data)

initial = 1000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

               High      Low     Open    Close   Volume  Adj Close
Date                                                              
2021-09-15  13360.0  13270.0  13320.0  13310.0   9740.0    13310.0
2021-09-16  13370.0  13270.0  13370.0  13270.0   5095.0    13270.0
2021-09-17  13390.0  13270.0  13270.0  13340.0   7600.0    13340.0
2021-09-21  13150.0  12970.0  13050.0  13020.0  39835.0    13020.0
2021-09-22  13010.0  12840.0  12960.0  13000.0  13138.0    13000.0
...             ...      ...      ...      ...      ...        ...
2022-09-08  14495.0  14430.0  14460.0  14440.0   7167.0    14440.0
2022-09-09  14585.0  14500.0  14540.0  14550.0   4700.0    14550.0
2022-09-12  14725.0  14635.0  14670.0  14710.0   8392.0    14710.0
2022-09-13  14845.0  14800.0  14845.0  14820.0   3573.0    14820.0
2022-09-14  14515.0  14375.0  14430.0  14390.0  12573.0    14390.0

[245 rows x 6 columns]


**MACD CROSS**

In [10]:
import BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    maximize='Equity Final [$]',
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 


if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACD.MACDCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True, #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

    #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)
    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        maximize='Equity Final [$]',
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   41.632653
Equity Final [$]                  976658.6885
Equity Peak [$]                  1044210.2985
Return [%]                          -2.334131
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -2.400014
Volatility (Ann.) [%]               11.004024
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.909993
Avg. Drawdown [%]                   -4.567711
Max. Drawdown Duration      168 days 00:00:00
Avg. Drawdown Duration       66 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                       8.109186
Worst Trade [%]                     -4.194151
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,70,69,81,14129.59700,14005.0,-8721.79000,-0.008818,2021-12-27,2022-01-17,21 days
1,71,94,106,13777.86450,13200.0,-41028.37950,-0.041942,2022-02-03,2022-02-22,19 days
2,72,121,140,13194.99350,14265.0,77040.46800,0.081092,2022-03-16,2022-04-13,28 days
3,70,147,148,14606.94825,14200.0,-28486.37750,-0.027860,2022-04-22,2022-04-25,3 days
4,71,170,181,13953.73075,13400.0,-39314.88325,-0.039683,2022-05-31,2022-06-15,15 days
5,70,191,230,13637.17150,14350.0,49897.99500,0.052271,2022-06-29,2022-08-25,57 days
6,67,243,244,14918.48275,14430.0,-32728.34425,-0.032743,2022-09-13,2022-09-14,1 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    42.44898
Equity Final [$]                   955691.233
Equity Peak [$]                   1013997.415
Return [%]                          -4.430877
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -4.554546
Volatility (Ann.) [%]               11.223102
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -9.162208
Avg. Drawdown [%]                   -5.991317
Max. Drawdown Duration      146 days 00:00:00
Avg. Drawdown Duration       87 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                       5.930866
Worst Trade [%]                     -3.913812
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,69,71,81,14340.63650,14005.0,-23158.9185,-0.023405,2021-12-29,2022-01-17,19 days
1,71,97,106,13737.66650,13200.0,-38174.3215,-0.039138,2022-02-08,2022-02-22,14 days
2,68,122,149,13642.19625,14275.0,43030.6550,0.046386,2022-03-17,2022-04-26,40 days
3,70,171,182,14009.00300,13495.0,-35980.2100,-0.036691,2022-06-01,2022-06-16,15 days
4,69,192,194,13576.87450,13390.0,-12894.3405,-0.013764,2022-06-30,2022-07-04,4 days
5,69,196,233,13466.33000,14265.0,55108.2300,0.059309,2022-07-06,2022-08-30,55 days
6,66,243,244,14918.48275,14430.0,-32239.8615,-0.032743,2022-09-13,2022-09-14,1 days


<Strategy MACDCross(MACDshort=15,MACDlong=40,MACDsignal=10)>

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   84.489796
Equity Final [$]                 924384.78225
Equity Peak [$]                  1054828.4945
Return [%]                          -7.561522
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                    -7.76895
Volatility (Ann.) [%]               16.296239
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -13.091661
Avg. Drawdown [%]                   -5.848542
Max. Drawdown Duration      168 days 00:00:00
Avg. Drawdown Duration       51 days 00:00:00
# Trades                                   14
Win Rate [%]                        35.714286
Best Trade [%]                       8.109186
Worst Trade [%]                     -4.194151
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-71,38,69,13990.40300,14060.0,-4941.38700,-0.004975,2021-11-11,2021-12-27,46 days
1,70,69,81,14129.59700,14005.0,-8721.79000,-0.008818,2021-12-27,2022-01-17,21 days
2,-70,81,94,13935.67525,13710.0,15797.26750,0.016194,2022-01-17,2022-02-03,17 days
3,72,94,106,13777.86450,13200.0,-41606.24400,-0.041942,2022-02-03,2022-02-22,19 days
4,-73,106,121,13134.66000,13130.0,340.18000,0.000355,2022-02-22,2022-03-16,22 days
5,72,121,140,13194.99350,14265.0,77040.46800,0.081092,2022-03-16,2022-04-13,28 days
6,-73,140,147,14194.38825,14535.0,-24864.65775,-0.023996,2022-04-13,2022-04-22,9 days
7,69,147,148,14606.94825,14200.0,-28079.42925,-0.027860,2022-04-22,2022-04-25,3 days
8,-69,148,170,14129.71000,13885.0,16884.99000,0.017319,2022-04-25,2022-05-31,36 days
9,71,170,181,13953.73075,13400.0,-39314.88325,-0.039683,2022-05-31,2022-06-15,15 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   71.020408
Equity Final [$]                  904662.2635
Equity Peak [$]                   1038838.349
Return [%]                          -9.533774
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -9.792379
Volatility (Ann.) [%]               15.501922
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -12.915973
Avg. Drawdown [%]                   -5.589694
Max. Drawdown Duration      230 days 00:00:00
Avg. Drawdown Duration       85 days 00:00:00
# Trades                                   13
Win Rate [%]                        30.769231
Best Trade [%]                       5.930866
Worst Trade [%]                     -4.583584
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,69,71,81,14340.63650,14005.0,-23158.91850,-0.023405,2021-12-29,2022-01-17,19 days
1,-70,81,97,13935.67525,13670.0,18597.26750,0.019064,2022-01-17,2022-02-08,22 days
2,72,97,106,13737.66650,13200.0,-38711.98800,-0.039138,2022-02-08,2022-02-22,14 days
3,-72,106,122,13134.66000,13575.0,-31704.48000,-0.033525,2022-02-22,2022-03-17,23 days
4,67,122,149,13642.19625,14275.0,42397.85125,0.046386,2022-03-17,2022-04-26,40 days
5,-68,149,171,14204.33875,13940.0,17975.03500,0.018610,2022-04-26,2022-06-01,36 days
6,70,171,182,14009.00300,13495.0,-35980.21000,-0.036691,2022-06-01,2022-06-16,15 days
7,-70,182,192,13428.19975,13510.0,-5726.01750,-0.006092,2022-06-16,2022-06-30,14 days
8,69,192,194,13576.87450,13390.0,-12894.34050,-0.013764,2022-06-30,2022-07-04,4 days
9,-69,194,196,13323.71950,13400.0,-5263.35450,-0.005725,2022-07-04,2022-07-06,2 days


<Strategy MACDCross_WithShortPosition(MACDshort=15,MACDlong=40,MACDsignal=10)>

**EMA CROSS**

In [11]:
import BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") 

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    EMAshort=range(10, 100, 10),\
    EMAlong=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.EMAlong > x.EMAshort),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstEMA.EmaCrossStrategy_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") 

    #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        EMAshort=range(10, 100, 10),\
        EMAlong=range(10, 200, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.EMAlong > x.EMAshort),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   60.816327
Equity Final [$]                 970410.74525
Equity Peak [$]                 1086343.07525
Return [%]                          -2.958925
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -3.042167
Volatility (Ann.) [%]               13.388496
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -14.212834
Avg. Drawdown [%]                   -3.253937
Max. Drawdown Duration      146 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                   11
Win Rate [%]                        36.363636
Best Trade [%]                       4.071313
Worst Trade [%]                     -5.043316
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,74,17,50,13365.83500,13910.0,40268.21000,0.040713,2021-10-12,2021-11-30,49 days
1,74,58,62,14034.12675,14005.0,-2155.37950,-0.002075,2021-12-10,2021-12-16,6 days
2,73,69,81,14129.59700,14005.0,-9095.58100,-0.008818,2021-12-27,2022-01-17,21 days
3,74,94,102,13777.86450,13720.0,-4281.97300,-0.004200,2022-02-03,2022-02-16,13 days
4,75,104,105,13566.82500,13310.0,-19261.87500,-0.018930,2022-02-18,2022-02-21,3 days
5,73,122,150,13642.19625,13755.0,8234.67375,0.008269,2022-03-17,2022-04-27,41 days
6,72,170,180,13953.73075,13250.0,-50668.61400,-0.050433,2022-05-31,2022-06-14,14 days
7,70,191,194,13637.17150,13390.0,-17302.00500,-0.018125,2022-06-29,2022-07-04,5 days
8,70,196,197,13466.33000,13510.0,3056.90000,0.003243,2022-07-06,2022-07-07,1 days
9,68,198,233,13767.81500,14265.0,33808.58000,0.036112,2022-07-08,2022-08-30,53 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   15.102041
Equity Final [$]                  1041165.244
Equity Peak [$]                   1069245.244
Return [%]                           4.116524
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                    4.236597
Volatility (Ann.) [%]                 6.54383
Sharpe Ratio                         0.647419
Sortino Ratio                        0.917893
Calmar Ratio                         0.960993
Max. Drawdown [%]                    -4.40856
Avg. Drawdown [%]                   -1.520895
Max. Drawdown Duration       24 days 00:00:00
Avg. Drawdown Duration        9 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       4.125622
Worst Trade [%]                      4.125622
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,72,208,244,13858.2605,14430.0,41165.244,0.041256,2022-07-25,2022-09-14,51 days


<Strategy EmaCrossStrategy(EMAshort=10,EMAlong=175)>

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   95.510204
Equity Final [$]                   854906.166
Equity Peak [$]                 1055935.80525
Return [%]                         -14.509383
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                  -14.891436
Volatility (Ann.) [%]               17.464811
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.366324
Avg. Drawdown [%]                   -4.134388
Max. Drawdown Duration      230 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                   22
Win Rate [%]                        27.272727
Best Trade [%]                       4.071313
Worst Trade [%]                     -5.043316
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-76,11,17,13065.00650,13300.0,-17859.50600,-0.017986,2021-10-04,2021-10-12,8 days
1,73,17,50,13365.83500,13910.0,39724.04500,0.040713,2021-10-12,2021-11-30,49 days
2,-73,50,58,13841.14550,13965.0,-9041.37850,-0.008948,2021-11-30,2021-12-10,10 days
3,72,58,62,14034.12675,14005.0,-2097.12600,-0.002075,2021-12-10,2021-12-16,6 days
4,-72,62,69,13935.67525,14060.0,-8951.38200,-0.008921,2021-12-16,2021-12-27,11 days
5,70,69,81,14129.59700,14005.0,-8721.79000,-0.008818,2021-12-27,2022-01-17,21 days
6,-71,81,94,13935.67525,13710.0,16022.94275,0.016194,2022-01-17,2022-02-03,17 days
7,73,94,102,13777.86450,13720.0,-4224.10850,-0.004200,2022-02-03,2022-02-16,13 days
8,-73,102,104,13652.08600,13500.0,11102.27800,0.011140,2022-02-16,2022-02-18,2 days
9,74,104,105,13566.82500,13310.0,-19005.05000,-0.018930,2022-02-18,2022-02-21,3 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   15.102041
Equity Final [$]                  1041165.244
Equity Peak [$]                   1069245.244
Return [%]                           4.116524
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                    4.236597
Volatility (Ann.) [%]                 6.54383
Sharpe Ratio                         0.647419
Sortino Ratio                        0.917893
Calmar Ratio                         0.960993
Max. Drawdown [%]                    -4.40856
Avg. Drawdown [%]                   -1.520895
Max. Drawdown Duration       24 days 00:00:00
Avg. Drawdown Duration        9 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       4.125622
Worst Trade [%]                      4.125622
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,72,208,244,13858.2605,14430.0,41165.244,0.041256,2022-07-25,2022-09-14,51 days


<Strategy EmaCrossStrategy_WithShortPosition(EMAshort=10,EMAlong=185)>

**SMA**

In [12]:
import BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    SMA_short=range(10, 100, 5),\
    SMA_long=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000,\
    constraint = lambda x: x.SMA_short < x.SMA_long)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMA.SmaCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        SMA_short=range(10, 100, 5),\
        SMA_long=range(10, 200, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000,\
        constraint = lambda x: x.SMA_short < x.SMA_long)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   36.734694
Equity Final [$]                  841208.8705
Equity Peak [$]                     1000000.0
Return [%]                         -15.879113
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                  -16.293682
Volatility (Ann.) [%]                8.640085
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.510828
Avg. Drawdown [%]                  -20.510828
Max. Drawdown Duration      253 days 00:00:00
Avg. Drawdown Duration      253 days 00:00:00
# Trades                                    5
Win Rate [%]                             20.0
Best Trade [%]                       3.456428
Worst Trade [%]                     -7.876262
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,69,74,86,14491.37900,13350.0,-78755.1510,-0.078763,2022-01-05,2022-01-24,19 days
1,64,127,155,14310.48800,13670.0,-40991.2320,-0.044757,2022-03-25,2022-05-10,46 days
2,60,177,186,14471.28000,13420.0,-63076.8000,-0.072646,2022-06-09,2022-06-22,13 days
3,58,205,241,14054.22575,14540.0,28174.9065,0.034564,2022-07-20,2022-09-09,51 days
4,58,244,244,14501.42850,14430.0,-4142.8530,-0.004926,2022-09-14,2022-09-14,0 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   20.408163
Equity Final [$]                1112769.51225
Equity Peak [$]                 1142799.51225
Return [%]                          11.276951
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   11.617189
Volatility (Ann.) [%]                8.222944
Sharpe Ratio                         1.412777
Sortino Ratio                         2.47666
Calmar Ratio                         3.652421
Max. Drawdown [%]                   -3.180682
Avg. Drawdown [%]                   -1.512436
Max. Drawdown Duration       14 days 00:00:00
Avg. Drawdown Duration        9 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       9.082037
Worst Trade [%]                      2.126055
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,75,183,222,13159.82025,14355.0,89638.48125,0.090820,2022-06-17,2022-08-15,59 days
1,77,235,244,14129.59700,14430.0,23131.03100,0.021261,2022-09-01,2022-09-14,13 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-73,53,74,13577.45725,14420.0,-61505.62075,-0.062055,2021-12-03,2022-01-05,33 days
1,64,74,86,14491.37900,13350.0,-73048.25600,-0.078763,2022-01-05,2022-01-24,19 days
2,-65,86,127,13283.91750,14240.0,-62145.36250,-0.071973,2022-01-24,2022-03-25,60 days
3,56,127,155,14310.48800,13670.0,-35867.32800,-0.044757,2022-03-25,2022-05-10,46 days
4,-56,155,177,13602.33350,14400.0,-44669.32400,-0.058642,2022-05-10,2022-06-09,30 days
5,49,177,186,14471.28000,13420.0,-51512.72000,-0.072646,2022-06-09,2022-06-22,13 days
6,-50,186,205,13353.57100,13985.0,-31571.45000,-0.047285,2022-06-22,2022-07-20,28 days
7,45,205,241,14054.22575,14540.0,21859.84125,0.034564,2022-07-20,2022-09-09,51 days
8,-45,241,244,14468.02700,14430.0,1711.21500,0.002628,2022-09-09,2022-09-14,5 days
9,45,244,244,14501.42850,14430.0,-3214.28250,-0.004926,2022-09-14,2022-09-14,0 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                        20.0
Equity Final [$]                 1136862.9025
Equity Peak [$]                  1167282.9025
Return [%]                           13.68629
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   14.103707
Volatility (Ann.) [%]                8.094605
Sharpe Ratio                         1.742359
Sortino Ratio                        3.422809
Calmar Ratio                         5.411905
Max. Drawdown [%]                   -2.606052
Avg. Drawdown [%]                   -1.206552
Max. Drawdown Duration       14 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                       8.010126
Worst Trade [%]                      2.271534
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,74,196,225,13466.33000,14545.0,79821.5800,0.080101,2022-07-06,2022-08-18,43 days
1,-74,225,237,14473.00225,14040.0,32042.1665,0.029918,2022-08-18,2022-09-05,18 days
2,78,237,244,14109.49800,14430.0,24999.1560,0.022715,2022-09-05,2022-09-14,9 days


<Strategy SmaCross_WithShortPosition(SMA_short=80,SMA_long=160)>

**ボリンジャーバンド**

In [13]:
import BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBD.BBsigma_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   38.367347
Equity Final [$]                   983175.996
Equity Peak [$]                   1037910.355
Return [%]                            -1.6824
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -1.730051
Volatility (Ann.) [%]               13.575423
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.803858
Avg. Drawdown [%]                   -4.165607
Max. Drawdown Duration      216 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                   19
Win Rate [%]                        31.578947
Best Trade [%]                       6.645103
Worst Trade [%]                     -4.103412
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,71,50,51,13978.85450,13650.0,-23348.66950,-0.023525,2021-11-30,2021-12-01,1 days
1,71,51,52,13717.56750,13580.0,-9767.29250,-0.010029,2021-12-01,2021-12-02,1 days
2,70,52,53,13647.22100,13645.0,-155.47000,-0.000163,2021-12-02,2021-12-03,1 days
3,70,53,74,13712.54275,14420.0,49522.00750,0.051592,2021-12-03,2022-01-05,33 days
4,75,86,87,13416.08250,13350.0,-4956.18750,-0.004926,2022-01-24,2022-01-25,1 days
5,75,87,88,13416.08250,13180.0,-17706.18750,-0.017597,2022-01-25,2022-01-26,1 days
6,75,88,89,13245.24100,13285.0,2981.92500,0.003002,2022-01-26,2022-01-27,1 days
7,74,89,90,13350.76075,13250.0,-7456.29550,-0.007547,2022-01-27,2022-01-28,1 days
8,74,90,108,13315.58750,13090.0,-16693.47500,-0.016942,2022-01-28,2022-02-25,28 days
9,73,108,115,13154.79550,12615.0,-39405.07150,-0.041034,2022-02-25,2022-03-08,11 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   79.591837
Equity Final [$]                   944250.006
Equity Peak [$]                 1105886.19675
Return [%]                          -5.574999
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -5.729633
Volatility (Ann.) [%]                16.84191
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.481977
Avg. Drawdown [%]                   -3.566597
Max. Drawdown Duration      216 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                   34
Win Rate [%]                        32.352941
Best Trade [%]                       6.645103
Worst Trade [%]                     -4.103412
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,71,50,51,13978.85450,13650.0,-23348.66950,-0.023525,2021-11-30,2021-12-01,1 days
1,71,51,52,13717.56750,13580.0,-9767.29250,-0.010029,2021-12-01,2021-12-02,1 days
2,70,52,53,13647.22100,13645.0,-155.47000,-0.000163,2021-12-02,2021-12-03,1 days
3,70,53,74,13712.54275,14420.0,49522.00750,0.051592,2021-12-03,2022-01-05,33 days
4,-70,74,75,14348.62100,14195.0,10753.47000,0.010706,2022-01-05,2022-01-06,1 days
5,-72,75,86,14124.73475,13350.0,55780.90200,0.054850,2022-01-06,2022-01-24,18 days
6,80,86,87,13416.08250,13350.0,-5286.60000,-0.004926,2022-01-24,2022-01-25,1 days
7,80,87,88,13416.08250,13180.0,-18886.60000,-0.017597,2022-01-25,2022-01-26,1 days
8,79,88,89,13245.24100,13285.0,3140.96100,0.003002,2022-01-26,2022-01-27,1 days
9,79,89,90,13350.76075,13250.0,-7960.09925,-0.007547,2022-01-27,2022-01-28,1 days


**RSI**

In [14]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   55.102041
Equity Final [$]                 1077218.7595
Equity Peak [$]                  1092388.7595
Return [%]                           7.721876
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                    7.951051
Volatility (Ann.) [%]               16.475079
Sharpe Ratio                         0.482611
Sortino Ratio                        0.731858
Calmar Ratio                         0.711355
Max. Drawdown [%]                  -11.177339
Avg. Drawdown [%]                   -4.960447
Max. Drawdown Duration      146 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       7.728745
Worst Trade [%]                      7.728745
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,74,91,225,13501.50325,14545.0,77218.7595,0.077287,2022-01-31,2022-08-18,199 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   39.183673
Equity Final [$]                1195729.07225
Equity Peak [$]                 1195729.07225
Return [%]                          19.572907
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   20.185166
Volatility (Ann.) [%]               15.440736
Sharpe Ratio                         1.307267
Sortino Ratio                        2.363007
Calmar Ratio                         1.965248
Max. Drawdown [%]                  -10.271053
Avg. Drawdown [%]                   -2.169448
Max. Drawdown Duration       96 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                    8
Win Rate [%]                             62.5
Best Trade [%]                       7.859022
Worst Trade [%]                     -2.474635
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,73,52,54,13647.22100,13675.0,2027.86700,0.002036,2021-12-02,2021-12-06,4 days
1,72,54,74,13742.69125,14420.0,48766.23000,0.049285,2021-12-06,2022-01-05,30 days
2,77,91,109,13501.50325,13190.0,-23985.75025,-0.023072,2022-01-31,2022-02-28,28 days
3,77,109,117,13255.29050,13035.0,-16962.36850,-0.016619,2022-02-28,2022-03-10,10 days
4,77,117,125,13099.52325,14055.0,73571.70975,0.072940,2022-03-10,2022-03-23,13 days
5,79,160,167,13622.09725,13285.0,-26630.68275,-0.024746,2022-05-17,2022-05-26,9 days
6,79,167,177,13350.76075,14400.0,82889.90075,0.078590,2022-05-26,2022-06-09,14 days
7,85,187,206,13310.56275,13970.0,56052.16625,0.049542,2022-06-23,2022-07-21,28 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-72,21,39,13701.83850,14120.0,-30107.62800,-0.030519,2021-10-18,2021-11-12,25 days
1,-69,39,51,14050.10600,13650.0,27607.31400,0.028477,2021-11-12,2021-12-01,19 days
2,72,51,53,13717.56750,13645.0,-5224.86000,-0.005290,2021-12-01,2021-12-03,2 days
3,72,53,65,13712.54275,13585.0,-9183.07800,-0.009301,2021-12-03,2021-12-21,18 days
4,72,65,74,13652.24575,14420.0,55278.30600,0.056236,2021-12-21,2022-01-05,15 days
5,-72,74,86,14348.62100,13350.0,71900.71200,0.069597,2022-01-05,2022-01-24,19 days
6,82,86,107,13416.08250,13045.0,-30428.76500,-0.027660,2022-01-24,2022-02-24,31 days
7,82,107,115,13109.57275,12615.0,-40554.96550,-0.037726,2022-02-24,2022-03-08,12 days
8,81,115,126,12677.44425,13970.0,104697.01575,0.101957,2022-03-08,2022-03-24,16 days
9,-82,126,138,13900.84850,14355.0,-37240.42300,-0.032671,2022-03-24,2022-04-11,18 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   88.571429
Equity Final [$]                1367161.12175
Equity Peak [$]                 1411811.12175
Return [%]                          36.716112
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   37.943189
Volatility (Ann.) [%]               24.619685
Sharpe Ratio                         1.541173
Sortino Ratio                        3.314053
Calmar Ratio                         3.865952
Max. Drawdown [%]                   -9.814707
Avg. Drawdown [%]                    -2.40122
Max. Drawdown Duration       68 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                      10.417017
Worst Trade [%]                     -3.153467
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-71,28,34,13930.70000,14370.0,-31190.30000,-0.031535,2021-10-27,2021-11-05,9 days
1,-67,34,88,14298.86850,13180.0,74964.18950,0.078249,2021-11-05,2022-01-26,82 days
2,78,88,130,13245.24100,14625.0,107621.20200,0.104170,2022-01-26,2022-03-30,63 days
3,-79,130,158,14552.60625,13250.0,102905.89375,0.089510,2022-03-30,2022-05-13,44 days
4,94,158,225,13315.58750,14545.0,115564.77500,0.092329,2022-05-13,2022-08-18,97 days
5,-94,225,227,14473.00225,14500.0,-2537.78850,-0.001865,2022-08-18,2022-08-22,4 days
6,-94,227,244,14428.22500,14430.0,-166.85000,-0.000123,2022-08-22,2022-09-14,23 days


<Strategy RsiOscillator_WithShortPosition(upper_bound=75,lower_bound=25,rsi_window=10)>

**ドンチャンブレイクアウト**

In [18]:
import BacktestingStrategies.Strategy_Donchian as btstdnchn
importlib.reload(btstdnchn)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstdnchn.DnchnBreakout, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    dnchn_long = range(10, 50, 5),
    dnchn_short = range(10, 50, 5),
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.dnchn_long > x.dnchn_short),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstdnchn.DnchnBreakout_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        dnchn_long = range(10, 50, 5),
        dnchn_short = range(10, 50, 5),
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    41.22449
Equity Final [$]                 880551.00675
Equity Peak [$]                    1035401.68
Return [%]                         -11.944899
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                  -12.264355
Volatility (Ann.) [%]                9.226203
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.745262
Avg. Drawdown [%]                   -2.811041
Max. Drawdown Duration      293 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                       1.608705
Worst Trade [%]                     -8.439336
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,72,22,50,13868.31000,13910.0,3001.68000,0.003006,2021-10-19,2021-11-30,42 days
1,69,71,81,14340.63650,14005.0,-23158.91850,-0.023405,2021-12-29,2022-01-17,19 days
2,69,125,151,14124.57225,14090.0,-2385.48525,-0.002448,2022-03-23,2022-04-28,36 days
3,67,177,180,14471.28000,13250.0,-81825.76000,-0.084393,2022-06-09,2022-06-14,5 days
4,63,206,233,14039.15150,14265.0,14228.45550,0.016087,2022-07-21,2022-08-30,40 days
5,60,243,244,14918.48275,14430.0,-29308.96500,-0.032743,2022-09-13,2022-09-14,1 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   21.632653
Equity Final [$]                 951453.76825
Equity Peak [$]                   1014777.547
Return [%]                          -4.854623
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -4.989808
Volatility (Ann.) [%]                 7.11605
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -6.240164
Avg. Drawdown [%]                   -5.005089
Max. Drawdown Duration      146 days 00:00:00
Avg. Drawdown Duration      126 days 00:00:00
# Trades                                    4
Win Rate [%]                             50.0
Best Trade [%]                       1.390972
Worst Trade [%]                     -3.356333
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,69,74,81,14491.37900,14005.0,-33560.15100,-0.033563,2022-01-05,2022-01-17,12 days
1,68,126,151,14039.15150,14090.0,3457.69800,0.003622,2022-03-24,2022-04-28,35 days
2,68,217,233,14069.30000,14265.0,13307.60000,0.013910,2022-08-05,2022-08-30,25 days
3,65,243,244,14918.48275,14430.0,-31751.37875,-0.032743,2022-09-13,2022-09-14,1 days


<Strategy DnchnBreakout(dnchn_long=35,dnchn_short=10)>

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   91.020408
Equity Final [$]                 851520.38325
Equity Peak [$]                 1050286.35525
Return [%]                         -14.847962
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                  -15.238112
Volatility (Ann.) [%]               17.253219
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -21.227161
Avg. Drawdown [%]                   -4.334815
Max. Drawdown Duration      190 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                                   21
Win Rate [%]                        52.380952
Best Trade [%]                       3.461096
Worst Trade [%]                    -12.114746
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,72,22,50,13868.31000,13910.0,3001.68000,0.003006,2021-10-19,2021-11-30,42 days
1,-72,50,51,13841.14550,13650.0,13762.47600,0.013810,2021-11-30,2021-12-01,1 days
2,-74,51,65,13582.43250,13585.0,-189.99500,-0.000189,2021-12-01,2021-12-21,20 days
3,-75,65,81,13517.75425,14005.0,-36543.43125,-0.036045,2021-12-21,2022-01-17,27 days
4,-70,81,84,13935.67525,13630.0,21397.26750,0.021935,2022-01-17,2022-01-20,3 days
5,-73,84,86,13562.53150,13350.0,15514.79950,0.015670,2022-01-20,2022-01-24,4 days
6,-76,86,88,13283.91750,13180.0,7897.73000,0.007823,2022-01-24,2022-01-26,2 days
7,-78,88,90,13114.75900,13250.0,-10548.79800,-0.010312,2022-01-26,2022-01-28,2 days
8,-76,90,107,13184.41250,13045.0,10595.35000,0.010574,2022-01-28,2022-02-24,27 days
9,-78,107,108,12980.42725,13090.0,-8546.67450,-0.008441,2022-02-24,2022-02-25,1 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   79.591837
Equity Final [$]                 898163.81875
Equity Peak [$]                  1069931.0075
Return [%]                         -10.183618
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                  -10.458811
Volatility (Ann.) [%]               17.803241
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -18.787865
Avg. Drawdown [%]                   -7.030295
Max. Drawdown Duration      190 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   11
Win Rate [%]                        54.545455
Best Trade [%]                       3.034152
Worst Trade [%]                     -9.571468
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-72,50,51,13841.14550,13650.0,13762.47600,0.013810,2021-11-30,2021-12-01,1 days
1,-74,51,86,13582.43250,13350.0,17200.00500,0.017113,2021-12-01,2022-01-24,54 days
2,-77,86,88,13283.91750,13180.0,8001.64750,0.007823,2022-01-24,2022-01-26,2 days
3,-79,88,90,13114.75900,13250.0,-10684.03900,-0.010312,2022-01-26,2022-01-28,2 days
4,-77,90,108,13184.41250,13090.0,7269.76250,0.007161,2022-01-28,2022-02-25,28 days
5,-79,108,116,13025.20450,12630.0,31221.15550,0.030342,2022-02-25,2022-03-09,12 days
6,-84,116,156,12567.48150,13535.0,-81271.55400,-0.076986,2022-03-09,2022-05-11,63 days
7,-73,156,158,13468.00175,13250.0,15914.12775,0.016187,2022-05-11,2022-05-13,2 days
8,-75,158,166,13184.41250,13245.0,-4544.06250,-0.004595,2022-05-13,2022-05-25,12 days
9,-75,166,184,13179.43725,13235.0,-4167.20625,-0.004216,2022-05-25,2022-06-20,26 days


<Strategy DnchnBreakout_WithShortPosition(dnchn_long=35,dnchn_short=25)>